In [ ]:
!pip install google-genai

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai

# Initialize the Google GenAI client
# It automatically picks up the GEMINI_API_KEY environment variable
client = genai.Client()

# Set the model we want our agents to use
MODEL_ID = 'gemini-2.5-flash'

def task_a_extractor_agent(raw_data: str) -> str:
    """Subagent A: Extracts key entities from raw data."""
    print("-> [Task A] Extracting data...")
    prompt = f"Extract only the main subjects and verbs from this text:\n\n{raw_data}"
    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

def task_b_formatter_agent(extracted_data: str) -> str:
    """Subagent B: Formats the extracted data into a neat list."""
    print("-> [Task B] Formatting data...")
    prompt = f"Format the following raw extracted data into a clean, bulleted Markdown list:\n\n{extracted_data}"
    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

def task_c_summarizer_agent(formatted_data: str) -> str:
    """Subagent C: Adds a final summary to the formatted list."""
    print("-> [Task C] Adding final summary...")
    prompt = f"Add a brief, one-sentence summary to the top of this list:\n\n{formatted_data}"
    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

def sequential_workflow_agent(user_prompt: str) -> str:
    """
    The orchestrator agent.
    It follows predefined, linear logic without asking an LLM for routing.
    """
    print("Starting Sequential Workflow...")

    # 1. Pass the user prompt to Task A
    result_a = task_a_extractor_agent(user_prompt)

    # 2. Pass Task A's output directly to Task B
    result_b = task_b_formatter_agent(result_a)

    # 3. Pass Task B's output directly to Task C
    final_result = task_c_summarizer_agent(result_b)

    # Return the final chain result to the user
    return final_result

# --- Execution ---
if __name__ == "__main__":
    # The user's initial input
    raw_user_input = (
        "In 2024, the company saw a massive surge in revenue up to $50 million. "
        "However, operating costs also skyrocketed to $40 million due to supply chain issues. "
        "The CEO, Jane Doe, announced a new restructuring plan for 2025 to optimize margins."
    )

    # Trigger the sequential agent
    final_output = sequential_workflow_agent(raw_user_input)

    print("\n=== Final Response to User ===")
    print(final_output)

Starting Sequential Workflow...
-> [Task A] Extracting data...
-> [Task B] Formatting data...
-> [Task C] Adding final summary...

=== Final Response to User ===
The company's CEO made an announcement amidst skyrocketing operating costs.

*   **company** saw
*   **operating costs** skyrocketed
*   **CEO** announced
